In [13]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\EL7md\anaconda3\python.exe -m pip install --upgrade pip' command.


In [11]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import datetime
import xgboost as xgb 
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import pickle


In [37]:
# load data from database
engine=create_engine('sqlite:///LogWeeklyData.db')
df=pd.read_sql_table("LogWeeklyData", con=engine)

In [38]:
# print the head
df.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Timestamp
0,5.707110,5.707110,5.707110,5.707110,0.009950,1.386294,5.707110,2014-12-01
1,5.988961,5.988961,5.988961,5.988961,0.009950,1.605430,5.988961,2014-12-15
2,5.846439,5.846439,5.831882,5.831882,1.098612,6.530878,5.839187,2014-12-22
3,5.888878,5.888878,5.888878,5.888878,0.009950,1.526056,5.888878,2015-01-12
4,5.564520,5.564520,5.564520,5.564520,0.693147,5.564520,5.564520,2015-01-19


In [39]:
#print shape
df.shape

(195, 8)

In [40]:
# split the data
date=datetime.datetime(2017, 12, 31)

train_data=df.loc[df['Timestamp']<=date]


test_data=df.loc[df['Timestamp']>date]





In [43]:
# create date features
def create_features(df):
    df['dayofmonth'] = df['Timestamp'].dt.day
    df['month'] = df['Timestamp'].dt.month
    df['year'] = df['Timestamp'].dt.year
    X = df[['dayofmonth','month','year']]
    Y= df['Weighted_Price']
    return X,Y


In [44]:
# split the data
X_train,y_train=create_features(train_data)
X_test,y_test=create_features(test_data)

C:\Users\EL7md\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\EL7md\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\EL7md\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [46]:
# check shape of x_train and x_test
print(X_train.shape)
print(X_test.shape)

(149, 3)
(46, 3)


## Model Building

In [47]:
# create an xgboost regression model
model = xgb.XGBRegressor()

# fit the model
model.fit(X_train,y_train)



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [48]:
# test the model
test_data['WeightedPricePreds']=model.predict(X_test)
test_data['ActualWeightedPrice']=y_test
mean_squared_error(test_data['ActualWeightedPrice'],test_data['WeightedPricePreds'])

C:\Users\EL7md\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\EL7md\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2.4350426134568854

In [58]:
# model predictions
data = {'dayofmonth':[12],
        'month':[1],
        'year':[2018]}
 
df_test=pd.DataFrame(data)
model.predict(df_test)

array([6.9967585], dtype=float32)

In [59]:
# build a linear regression model
lrmodel=linear_model.LinearRegression()
lrmodel.fit(X_train,y_train)

LinearRegression()

In [60]:
# test the model
mean_squared_error(y_test,model.predict(X_test))

2.4350426134568854

> Both models have the same mean squared error

In [61]:
# save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(lrmodel, f)

In [62]:
# save the first model
with open('model1.pkl', 'wb') as f:
    pickle.dump(model, f)